In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import numpy as np
import os

In [33]:
def load_sentences(filepath):
    def special_chars_to_words(sentence):
        special_char_dict = {
            "0": "nol",
            "1": "satu",
            "2": "dua",
            "3": "tiga",
            "4": "empat",
            "5": "lima",
            "6": "enam",
            "7": "tujuh",
            "8": "delapan",
            "9": "sembilan",
            "\n": " newline "
        }
        
        new_sentence = ""
        for c in sentence:
            if c in special_char_dict.keys():
                new_sentence += special_char_dict[c]
            else:
                new_sentence += c
                
        return new_sentence
        
    VALID_CHARACTERS = string.ascii_letters + " -\n" + string.digits
    SPACEABLE_CHARACTER = "-"
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.read().strip() + "\n\n"
            
        # Remove invalid characters
        modified_content = "".join([c for c in content if c in VALID_CHARACTERS])
        
        # Change spaceable character to space
        modified_content = "".join([" " if c in SPACEABLE_CHARACTER else c for c in modified_content])
        
        # Change special characters to words
        modified_content = special_chars_to_words(modified_content)
        
        # Remove double spaces
        modified_content = modified_content.replace("  ", " ")
        
        """
        # Get nonzero and non-one-word bait as sentence
        sentences = modified_content.split("\n\n")
        sentences = list(map(lambda sentence: (sentence + ("" if sentence.endswith("\n") else "\n")).replace("\n", " "), sentences))
        sentences = list(filter(lambda sentence: sentence != "", sentences))
        sentences = list(filter(lambda sentence: " " in sentence.strip(), sentences)) # Jika tidak ada spasi, berarti satu kata
        """
        
        # return sentences
        return [modified_content]
    
    except FileNotFoundError:
        print("Warning: File not found")
        return []

In [54]:

my_sentences = []
PATH = "data/chairil-anwar/"
for file in os.listdir(PATH):
    if file.endswith(".txt"):
        print("Loaded {}".format(file))
        my_sentences += load_sentences(PATH + file)

# my_sentences = load_sentences("data/chairil-anwar/puisi-kehidupan.txt")

Loaded aku.txt
Loaded cerita-buat-dien-tamaela.txt
Loaded cinta-dan-benci.txt
Loaded cintaku-jauh-di-pulau.txt
Loaded derai-derai-cemara.txt
Loaded di-mesjid.txt
Loaded diponegoro.txt
Loaded doa.txt
Loaded hampa.txt
Loaded kawanku-dan-aku.txt
Loaded kepada-kawan.txt
Loaded kepada-peminta-minta.txt
Loaded krawang-bekasi.txt
Loaded lagu-siul.txt
Loaded nisan.txt
Loaded persetujuan-dengan-bung-karno.txt
Loaded prajurit-jaga-malam.txt
Loaded puisi-kehidupan.txt
Loaded rumahku.txt
Loaded sajak-putih.txt
Loaded sebuah-kamar.txt
Loaded selamat-tinggal.txt
Loaded senja-di-pelabuhan-kecil.txt
Loaded sia-sia.txt
Loaded tak-sepadan.txt
Loaded tuti-artic.txt
Loaded yang-terampas-dan-yang-terputus.txt


In [55]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(my_sentences)
total_words = len(tokenizer.word_index) + 1 # Kata kosong termasuk (yaitu token 0)

In [56]:
input_sequences = []
tokens = tokenizer.texts_to_sequences(my_sentences)
for token in tokens:
    for i in range(1, len(token)):
        n_gram_sequence = token[:i+1]
        input_sequences.append(n_gram_sequence)

In [57]:
max_sequence_len = max([len(sequence) for sequence in input_sequences])
max_sequence_len

220

In [58]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = "pre"))

In [59]:
xs = input_sequences[:,:-1]

labels = input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [61]:
model = Sequential([
    Embedding(total_words, 128, input_length = max_sequence_len - 1),
    Bidirectional(LSTM(128)),
    Dense(total_words, activation = "softmax")
])
adam = Adam(lr = 0.01) # lr: learning rate
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
history = model.fit(xs, ys, epochs=10, verbose=1)

Epoch 1/10
2583/2583 [==============================] - 127s 49ms/sample - loss: 6.3626 - acc: 0.1882
Epoch 2/10
2583/2583 [==============================] - 126s 49ms/sample - loss: 5.4434 - acc: 0.2168
Epoch 3/10
2583/2583 [==============================] - 131s 51ms/sample - loss: 4.9135 - acc: 0.2191
Epoch 4/10
2583/2583 [==============================] - 131s 51ms/sample - loss: 4.3448 - acc: 0.2408
Epoch 5/10
2583/2583 [==============================] - 136s 53ms/sample - loss: 3.6567 - acc: 0.2946
Epoch 6/10
2583/2583 [==============================] - 141s 54ms/sample - loss: 2.8892 - acc: 0.3914
Epoch 7/10
1824/2583 [====================>.........] - ETA: 44s - loss: 2.0650 - acc: 0.5389

In [53]:
seed_text = "masihkah aku diberi kesempatan".lower()
bait = 0
word_count = 0
already_newline = False
already_bait_baru = False

while bait < 8 and word_count < 1000:
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen = max_sequence_len - 1, padding = "pre")
    predicted = model.predict_classes(token_list, verbose = 0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    if output_word == "newline":
        if already_newline and (not already_bait_baru):
            bait += 1
            already_bait_baru = True
            print("Bait {} selesai".format(bait))
        already_newline = True
    else:
        already_bait_baru = False
        already_newline = False
        
    seed_text += " " + output_word
    word_count += 1
    print(".", end="")
    
print()
print(seed_text.replace("newline", "\n"))

.Bait 1 selesai
..............................................................................Bait 2 selesai
....Bait 3 selesai
...................................Bait 4 selesai
....Bait 5 selesai
..............................................................Bait 6 selesai
...Bait 7 selesai
..........................................................................Bait 8 selesai
.
masihkah aku diberi kesempatan 
 
 hari ini aku menuju satu puncak tangga yang baru 
 karena aku akan membuka lembaran baru 
 untuk sisa jatah umurku yang baru 
 daun gugur satu satu 
 semua terjadi karena ijin allah 
 tapi coba aku tengok kebelakang 
 ternyata aku masih banyak berhutang 
 ya berhutang pada diriku 
 karena ibadahku masih pas pasan 
 kuraba dahiku 
 astagfirullah sujudku masih jauh dari khusyuk 
 kutimbang keinginanku 
 hmm masih lebih besar duniawiku 
 
 ya allah 
 
 akankah aku masih bertemu tanggal dan bulan yang sama di tahun depan 
 akankah aku masih merasakan rasa ini pada tanggal dan bul

In [60]:
model.save_weights("./data/checkpoints/model-checkpoint-with-newline-one-data")

In [28]:
model.load_weights("./data/checkpoints/model-checkpoint")

In [48]:
sorted(list(tokenizer.word_index.keys()))

['adalah',
 'airmataku',
 'akan',
 'akankah',
 'akhirat',
 'aku',
 'allah',
 'astagfirullah',
 'banyak',
 'baru',
 'berhutang',
 'bertambah',
 'bertemu',
 'besar',
 'bisa',
 'bulan',
 'coba',
 'dahiku',
 'dalam',
 'dan',
 'dari',
 'daun',
 'depan',
 'di',
 'diberi',
 'diriku',
 'dunia',
 'duniawiku',
 'engkau',
 'gugur',
 'hamba',
 'hambamu',
 'hari',
 'hmm',
 'ibadah',
 'ibadahku',
 'ijikanlah',
 'ijin',
 'ijinkan',
 'ingin',
 'ini',
 'jatah',
 'jauh',
 'jika',
 'karena',
 'kebelakang',
 'keinginanku',
 'kelemahanku',
 'kesempatan',
 'khalifahmu',
 'khusyuk',
 'kuraba',
 'kutimbang',
 'lebih',
 'lembaran',
 'lewat',
 'masih',
 'masihkah',
 'melihat',
 'membuka',
 'mendalam',
 'menuju',
 'merasakan',
 'mulai',
 'newline',
 'pada',
 'pas',
 'pasan',
 'pasti',
 'pelan',
 'penyesalanku',
 'puncak',
 'rasa',
 'sama',
 'sana',
 'sangat',
 'satu',
 'sebagai',
 'sedih',
 'sehingga',
 'seimbang',
 'sempurna',
 'semua',
 'senyummu',
 'sisa',
 'sujudku',
 'tahun',
 'tanda',
 'tangga',
 'tanggal'

In [51]:
print(my_sentences[0].replace("newline", "\n"))

Hari hari lewat pelan tapi pasti 
 
 Hari ini aku menuju satu puncak tangga yang baru 
 Karena aku akan membuka lembaran baru 
 Untuk sisa jatah umurku yang baru 
 Daun gugur satu satu 
 Semua terjadi karena ijin Allah 
 Umurku bertambah satu satu 
 Semua terjadi karena ijin Allah 
 Tapi coba aku tengok kebelakang 
 Ternyata aku masih banyak berhutang 
 Ya berhutang pada diriku 
 Karena ibadahku masih pas pasan 
 Kuraba dahiku 
 Astagfirullah sujudku masih jauh dari khusyuk 
 Kutimbang keinginanku 
 Hmm masih lebih besar duniawiku 
 
 Ya Allah 
 
 Akankah aku masih bertemu tanggal dan bulan yang sama di tahun depan 
 Akankah aku masih merasakan rasa ini pada tanggal dan bulan yang sama di tahun depan 
 Masihkah aku diberi kesempatan 
 
 Ya Allah 
 
 Tetes airmataku adalah tanda kelemahanku 
 Rasa sedih yang mendalam adalah penyesalanku 
 Astagfirullah 
 Jika Engkau ijinkan hamba bertemu tahun depan 
 Ijinkan hambaMU ini mulai hari ini lebih khusyuk dalam ibadah 
 Timbangan dunia dan ak